# Analyse TripAdvisor Hotelbewertungen (DuckDB + NLP)

## Datenquelle via DuckDB
Dieses Notebook kann den Datensatz direkt aus DuckDB lesen (empfohlen bei größeren CSVs).

In [ ]:

# Bootstrap: CSV -> DuckDB (einmalig)
from pathlib import Path
from src.data_store import bootstrap_from_csv, get_text_and_rating, DEFAULT_DB
csv_path = "data/tripadvisor_hotel_reviews.csv"
if not Path(DEFAULT_DB).exists():
    bootstrap_from_csv(csv_path)
df_raw = get_text_and_rating()
df_raw.head()


In [ ]:

# Weiterverarbeitung wie bisher
from src.preprocessing import clean_text
df = df_raw.copy()
df["clean_review"] = df.iloc[:,0].astype(str).apply(clean_text)
df.head()
